# Tradesman

Tradesman is a friendly model builder for transportation models. 

On this example, we show how to create a transportation model for Gibraltar, a small british territory in mainland Europe. In the end of the presetation, we also show some data visualization.

## Import package

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

from uuid import uuid4
from tradesman.model import Tradesman

c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\cg\alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\cg\alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\cg\alpha_shapes.


## Create model

In [2]:
from tempfile import gettempdir

model_place = "Gibraltar"
folder = os.path.join(gettempdir(), uuid4().hex)

We're going to save our model in the `"/tmp"` folder, but if you have a Google account and want to save your model into your Google Drive, you can run the following commands, accept the terms and conditions, and replace the folder path to `"/content/gdrive"`.

`from google.colab import drive`

`drive.mount("/content/gdrive")`

In [3]:
model = Tradesman(network_path=folder, model_place=model_place)


In [4]:
%%time
# Import the model area
model.import_model_area()
model.add_country_borders()
model.import_subdivisions()

CPU times: total: 46.9 ms
Wall time: 1.35 s


In [5]:
%%time
# Import the model network
model.import_network()

CPU times: total: 1.42 s
Wall time: 12.4 s


In [6]:
%%time
# Import population
model.import_population()

CPU times: total: 109 ms
Wall time: 234 ms


As Monaco has a really small territorial area, we'll create Traffic Analysis Zones (TAZs) more suitable sizes to the country rather than the default ones.

In [7]:
# Build TAZs
model.build_zoning(min_zone_pop=100, max_zone_pop=1500)

Expect 594 total hexbins for this bounding box


100%|██████████| 18/18 [00:00<00:00, 1867.04it/s]
c:\src\Tradesman-demo\.venv\lib\site-packages\tradesman\model_creation\zoning\zones_with_location.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gpd.GeoDataFrame(hexb[["hex_id"]], geometry=hexb.centroid, crs="EPSG:4326")
c:\src\Tradesman-demo\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\src\Tradesman-demo\.venv\lib\site-packages\tradesman\model_creation\zoning\create_clusters.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centr

Done 0/1 states


1it [00:00,  2.93it/s]
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 0.
  warnings.warn(message)
c:\src\Tradesman-demo\.venv\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
c:\src\Tradesman-demo\.venv\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected compon

In [8]:
%%time
# Import population pyramid
model.import_pop_by_sex_and_age()

CPU times: total: 1.42 s
Wall time: 4.54 s


In [9]:
%%time
# Import amenities
model.import_amenities()

CPU times: total: 46.9 ms
Wall time: 383 ms


In [10]:
# Import buildings
model.import_buildings(download_from_bing=False)

In case you don't want to change any of Tradesman's default configurations, you can simply replace the above cells with the following function:

`model.create()`

## Create a synthetic population

Let's create our synthetic population for Monaco! It's a two-step process: first, we sample a population dataset to create our seed sample that resembles Monaco (seed creation) and then, we run the synthesizer.

In [11]:
# Create sample to build synthetic population
model.build_population_synthesizer_data(sample_size=0.02)

Calculated vs Expected percentage of households per size in the sample

# Person      Calculated    Expected
----------  ------------  ----------
1 person           16.12       19.06
2-3 person         52.07       53.4
4-5 person         28.51       25.56
6+ person           3.31        1.98

Difference in calculated vs expected percentage of households in the sample

# Person      Diff.
----------  -------
1 person      -2.94
2-3 person    -1.33
4-5 person     2.95
6+ person      1.33



From the output, we can visualize if there are more or fewer households of certain categories. It's important to highlight that differences between expected and found values are expected because the data used to create the sample doesn't belong to Monaco.

In [12]:
# Create synthetic population
# model.synthesize_population()

After the execution of the above cell, there are two main outputs. The ones related to non-controlled or controlled variables. The first one represents the differences between expected and found values for non-controlled variables based on United Nation Household survey data. The graphs represent the percentage difference between the synthetic over the expected population for controlled variables.

## And as promised, a bit of data visualization

### Import libraries

In [13]:
import geopandas as gpd
import folium

### Layer set up

In [14]:
colors = ["#219EBC", "#ffb703", "#8ECAE6", "#023047", "#fb8500"]
location = [36.142260, -5.346601] # This is the location of the Monte Carlo Casino, in case you're curious

In [15]:
# Let's create a variable with our project database connection
cnx = model._project.conn

### Plot the network

In [16]:
query = "SELECT link_type, distance, modes, ST_AsBinary(geometry) geom FROM links;"
links = gpd.read_postgis(query, con=cnx, geom_col="geom", crs=4326)

# We'll plot only some link types
links = links[links.link_type.isin(["residential", "primary", "secondary", "tertiary"])]

In [17]:
m = None

for idx, tp in enumerate(links.link_type.unique()):
  gdf = links[links.link_type == tp]
  if m:
    gdf.explore(m=m, name=tp, tiles="CartoDB positron", tooltip=False, popup=True,
                  zoom_start=14, location=location, legend=False, color=colors[idx])
  else:
    m = gdf.explore(name=tp, tiles="CartoDB positron", tooltip=False, popup=True,
                      zoom_start=14, location=location, legend=False, color=colors[idx])

folium.LayerControl().add_to(m)

m

### Plot the populational density


In [18]:
query = "SELECT *, ST_AsBinary(geometry) geom FROM zones;"
zones = gpd.read_postgis(query, con=cnx, geom_col="geom", crs=4326)
zones.drop(columns=["geometry"], inplace=True)

# Let's create our populational densisity variable
zones["POP_DENSITY"] = zones["population"] / (zones["geom"].to_crs(3857).area * 10e-6)


In [24]:
zones.explore("POP_DENSITY", tiles="CartoDB positron", cmap="Blues", tooltip=False,
              style_kwds={"fillOpacity": 1.0}, zoom_start=14, location=location, popup=True)
